In [1]:
!pip3 install --upgrade pip
!pip3 install boto3
!pip3 install pyyaml

In [2]:
from functools import reduce
import os
import boto3
import yaml
from datetime import datetime, time, timedelta
import json
from pprint import pprint
from pprint import PrettyPrinter
from statistics import mean
import sys

In [ ]:
print(sys.version_info)
print(sys.version_info[:3])

In [ ]:
pp = PrettyPrinter() #indent=1)
print(pp)
section_delimiter = '\n{}\n'.format('--- '*20)

In [ ]:
config = yaml.safe_load(open('config.yaml', 'r'))
pp.pprint(config)
aws_region = config['aws']['regions'][0]
account = config['aws']['accounts'][0]['name']
profile = config['aws']['accounts'][0]['profile']
account_number = config['aws']['accounts'][0]['account-number']

print(section_delimiter)

print(('Selected Config: {} - {} - {} - {}'.format(str(account_number),account,profile,aws_region)))

In [ ]:
session = boto3.session.Session(profile_name=profile,region_name=aws_region)
print(('Session: {} - {}'.format(session.profile_name,session.region_name)))


In [ ]:
ec2Client = session.client('ec2')
cwClient = session.client('cloudwatch',region_name=aws_region)
print(ec2Client)
print(cwClient)

In [ ]:
ec2s = ec2Client.describe_instances(
    Filters=[
        {
            'Name': 'tag:owner',
            'Values': ['jimmyray']
        } 
    ]
)

pp.pprint(ec2s)

In [15]:
def get_metric(instanceId, stat, metric):
    data = cwClient.get_metric_statistics(
        Period=900,
        StartTime=(datetime.utcnow() - timedelta(days=14)),
        EndTime=datetime.utcnow(),
        MetricName=metric,
        Namespace='AWS/EC2',
        Statistics=[
            stat
        ],
        Dimensions=[
            {
                'Name':'InstanceId',
                'Value':instanceId
            }
        ]
    )
    
    return data

In [16]:
def get_images():
    response = ec2Client.describe_images(
        Owners=['self'],
        Filters=[
        {
            'Name': 'owner-id',
            'Values': [
                str(account_number),
            ]
        },
    ])['Images']
    
    data = {}
    for image in response:
        #print(image)
        data[image['ImageId']] = image
        
    return data

In [ ]:
imageData = get_images()
pp.pprint(imageData)


In [ ]:
stat='Average'
metric='CPUUtilization'
wantedTags=['name','owner']
na='N/A'

pp.pprint(ec2s['Reservations'])

print(section_delimiter)

print('# of EC2s: {}\n'.format(str(len(ec2s['Reservations']))))

for r in ec2s['Reservations']:
    for i in r['Instances']:
        tagStrings=[]
        print('[EC2]')
        print('Instanceid: {}'.format(i['InstanceId']))
        print('InstanceType: {}'.format(i['InstanceType']))
        print('Architecture: {}'.format(i['Architecture']))
        print('State: {}'.format(i.get('State').get('Name')))
        print()
        print('ImageId: {}'.format(i['ImageId']))
        
        if i['ImageId'] in imageData:
            print('ImageName: {}'.format(imageData[i['ImageId']].get('Name')))
            print('ImageDescription: {}'.format(imageData[i['ImageId']].get('Description')))
        
            past = datetime.strptime(imageData[i['ImageId']].get('CreationDate'),'%Y-%m-%dT%H:%M:%S.000Z')
            #print(past)
            present = datetime.now()
            #print(present)
            delta = present-past
            #print(delta.days)
        
            print('ImageAgeDays: {}'.format(str((present-past).days)))
            
        print('\nVpcId: {}'.format(i.get('VpcId', na)))
        print('SubnetId: {}\n'.
        format(i.get('SubnetId',na)))
        
        print('Tags:')
        
        for tag in i['Tags']:
            if tag['Key'] in wantedTags:
                tagStrings.append('Key: {}, Value: {}'.format(tag['Key'],tag['Value']))
                
        for tagString in sorted(tagStrings):
            print(tagString)
            
        print()
        
        data = get_metric(i['InstanceId'], stat, metric)

        stats=[]
        for dp in data['Datapoints']:
            stats.append(dp[stat])

        print('Average CPU Utilization: {} %'.format(round(mean(stats),2)))            
    
    print('\nEBS:')
    
    ec2 = session.resource('ec2')
    
    for bdm in i['BlockDeviceMappings']:
        volume = ec2.Volume(bdm['Ebs']['VolumeId'])
        print('Volume ID: {}'.format(volume.volume_id))
        print('Volume Size: {}GiBs'.format(volume.size))
        print('Volume Type: {}'.format(volume.volume_type))
        print('Device Mapping: {} delete on termination = {}'.format(bdm['DeviceName'],bdm['Ebs']['DeleteOnTermination']))

    
    print(section_delimiter)
                                

In [ ]:
sgs = ec2Client.describe_security_groups()
pp.pprint(sgs)

print(section_delimiter)

for group in sgs['SecurityGroups']:
    #pp.pprint(group)
    print("{} - {}".format(group['GroupId'],group['Description']))

print(section_delimiter)

sgs = ec2Client.describe_security_groups(GroupIds=['<SG_ID>'])

pp.pprint(sgs)
#print(sgs)

In [ ]:
sqclient = boto3.client('service-quotas')
response = sqclient.list_service_quotas(
    ServiceCode='ec2',
    # NextToken='string',
    # MaxResults=123
)

pp.pprint(response)